# Download SmartSeq2 Expression Matrix as an Input to Scanpy

Suppose I want to get a SmartSeq2 expression matrix that I can analyze using scanpy. How can I go about finding something like this using the DSS API?

In [139]:
import hca.dss
client = hca.dss.DSSClient()

Well, first things first: We're going to need to search for a `.results` file. This file should contain what we need to put together an expression matrix.

Here's our ElasticSearch query.

In [140]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "files.file_json.files.content.file_core.file_format": "results" # It needs to have a
                    }                                                                    # results file...
                },
                {
                    "range": {
                        "manifest.version": {
                            "gte": "2018-07-12T100000.000000Z" # ...and preferably not be too old, either.
                        }
                    }
                }
            ]
        }
    }
}

This query looks for bundles that satisfy a *bool*ean condition consisting of two checks, both of which *must* be true. The *match* check looks for the `file_format` field and returns true if its value matches the string 'results'. The second check, *range*, returns true if the bundle's `manifest.version` has a value greater than or equal to 7/12/18. In short, __this query will find bundles that contain a `.results` file and are newer than July 12, 2018.__

NOTE: Prior to HCA's official release, there are no analysis bundles in `production`. Instead, you will need to look under `integration` in order to find one. To do this, set `client.host`:

In [141]:
client.host = 'https://dss.integration.data.humancellatlas.org/v1'

Okay, now let's execute the search. Since the `files` section of the bundle is pretty long, I'll only print the portion containing a results file (in this case, the 28th file in the bundle). If you want, you can always print the entire bundle to get a better picture of where the file is located.

In [142]:
import json

# Print part of a recent analysis bundle with a results file

bundles = client.post_search(es_query=query, replica='aws', output_format='raw')
print(json.dumps(bundles['results'][0]['metadata']['files']['file_json']['files'][27], indent=4, sort_keys=True))

{
    "content": {
        "describedBy": "http://schema.integration.data.humancellatlas.org/type/file/5.2.1/analysis_file",
        "file_core": {
            "file_format": "results",
            "file_name": "4ee53206-80cd-4144-8c75-b62399a7ae99_rsem.genes.results"
        },
        "schema_type": "file"
    },
    "hca_ingest": {
        "document_id": "153552fc-d65f-41af-afa6-3fa85a87d24f",
        "submissionDate": "2018-07-25T21:39:23.972Z"
    }
}


Okay! It looks like we've found a file uuid we can use: `153552fc-d65f-41af-afa6-3fa85a87d24f`. Let's save it locally.

In [143]:
results = client.get_file(replica='aws', uuid='153552fc-d65f-41af-afa6-3fa85a87d24f')
open('matrix.results', 'w').write(results.decode("utf-8"))

7629174

Here's what our file, `matrix.results`, looks like. I've truncated the output so it doesn't take up too much room.

In [144]:
print(open('matrix.results', 'r').read()[:873])

gene_id	transcript_id(s)	length	effective_length	expected_count	TPM	FPKM	posterior_mean_count	posterior_standard_deviation_of_count	pme_TPM	pme_FPKM
ENSG00000000003.14	ENST00000373020.8,ENST00000494424.1,ENST00000496771.5,ENST00000612152.4,ENST00000614008.4	1749.40	1541.80	0.00	0.00	0.00	0.00	0.00	2.02	23.44
ENSG00000000005.5	ENST00000373031.4,ENST00000485971.1	940.50	732.90	0.00	0.00	0.00	0.00	0.00	1.54	17.88
ENSG00000000419.12	ENST00000371582.8,ENST00000371584.8,ENST00000371588.9,ENST00000413082.1,ENST00000466152.5,ENST00000494752.1	977.83	770.24	0.00	0.00	0.00	0.00	0.00	3.32	38.56
ENSG00000000457.13	ENST00000367770.5,ENST00000367771.10,ENST00000367772.8,ENST00000423670.1,ENST00000470238.1	3197.00	2989.40	0.00	0.00	0.00	0.00	0.00	0.86	9.99
ENSG00000000460.16	ENST00000286031.10,ENST00000359326.8,ENST00000413811.3,ENST00000459772.5,ENST00000466580.6,ENST0000047


For our matrix, however, we might only want _some_ of these values. In my case, suppose I only want the `gene_id` and `TPM` values. We can extract these values easily using Python's `csv` module.

In [145]:
import csv

# Take the data we want out of the results file and store it into a tsv file

with open('matrix.results', 'r') as infile, open('matrix.tsv', 'w', newline='') as outfile:
    reader = csv.DictReader(infile, delimiter='\t')
    writer = csv.DictWriter(outfile, fieldnames=['gene_id', 'TPM'], delimiter='\t')
    for row in reader:
        writer.writerow({'gene_id': row['gene_id'], 'TPM': row['TPM']})

Our new file, `matrix.tsv`, looks something like this:

In [146]:
print(open('matrix.tsv', 'r').read()[:214])

ENSG00000000003.14	0.00
ENSG00000000005.5	0.00
ENSG00000000419.12	0.00
ENSG00000000457.13	0.00
ENSG00000000460.16	0.00
ENSG00000000938.12	0.00
ENSG00000000971.15	0.00
ENSG00000001036.13	0.00
ENSG00000001084.10	0.00


Now that we have a file containing what we want, we can transpose it and read it into scanpy.

In [147]:
import scanpy.api as sc

adata = sc.read_csv(filename='matrix.tsv', delimiter='\t').transpose()

But how do we know that everything worked? Let's print our AnnData object (truncating the output again).

In [148]:
print(adata)
for i in range(0, 153):
    print('{:<6}'.format('{:.1f}'.format(adata.X[i])), end='' if (i + 1) % 17 != 0 else '\n' )

AnnData object with n_obs × n_vars = 1 × 58347 
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   


And just to make it easier to see the relevant data in our matrix...

In [149]:
for i in range(len(adata.X)-1):
    if adata.X[i] != 0:
        print(adata.X[i], end=' ')

309.74 2289.43 1079.51 33.39 71.25 306.59 55.48 262.12 233.33 359.17 579.74 875.4 1571.24 7501.46 114659.92 347.55 76.15 1258.66 42.63 170.67 1384.1 2636.91 8210.16 23152.72 40069.76 15311.09 51841.59 988.92 8740.85 1786.69 13603.32 33274.64 314.7 1665.7 6726.94 12717.9 46112.46 10746.5 2619.5 637.16 21969.11 17856.29 794.83 23246.24 2372.67 78258.91 26619.13 109.81 8582.35 1111.28 1170.91 6920.93 161.06 250.58 171.18 126.3 20162.46 118.0 6422.18 774.58 9770.59 9643.0 3571.98 36718.99 45083.26 1618.13 18952.62 24666.38 1357.03 1071.54 694.25 652.7 1615.82 386.99 5256.2 867.7 17468.06 199.05 841.76 99981.39 15283.49 127.55 858.33 220.5 1144.79 1475.82 9094.49 6.09 35948.45 183.31 12083.88 1119.97 561.17 766.69 147.05 1427.83 3711.39 305.53 279.72 2907.42 105.23 

Okay, so we have one AnnData object we can use with scanpy. __But what if we want a second one?__ Let's go through the steps again, this time with a different `.results` file. We can use the same query to get a bunch of analysis bundles, but this time get our `.results` file from the _second_ bundle.

In [150]:
bundles = client.post_search(es_query=query, replica='aws', output_format='raw')
print(json.dumps(bundles['results'][1]['metadata']['files']['file_json']['files'][27], indent=4, sort_keys=True))

{
    "content": {
        "describedBy": "https://schema.humancellatlas.org/type/file/5.2.1/analysis_file",
        "file_core": {
            "file_format": "results",
            "file_name": "84b516d0-30e9-4693-9658-a63aecb48040_rsem.genes.results"
        },
        "schema_type": "file"
    },
    "hca_ingest": {
        "document_id": "9ae45bb8-7130-4e06-a5af-87b587155034",
        "submissionDate": "2018-07-25T21:10:58.516Z"
    }
}


With the new uuid, we can get the `.results` file itself.

In [151]:
results2 = client.get_file(replica='aws', uuid='9ae45bb8-7130-4e06-a5af-87b587155034')
open('matrix2.results', 'w').write(results2.decode("utf-8"))

7629176

Again, let's take the data we want out of the results file and store it into a tsv file...

In [152]:
with open('matrix2.results', 'r') as infile, open('matrix2.tsv', 'w', newline='') as outfile:
    reader = csv.DictReader(infile, delimiter='\t')
    writer = csv.DictWriter(outfile, fieldnames=['gene_id', 'TPM'], delimiter='\t')
    for row in reader:
        writer.writerow({'gene_id': row['gene_id'], 'TPM': row['TPM']})

...and then create another AnnData object & print it.

In [153]:
adata2 = sc.read_csv( filename='matrix.tsv', delimiter='\t' ).transpose()

print(adata2)
for i in range(0, 153):
    print( '{:<6}'.format('{:.1f}'.format(adata2.X[i])), end='' if (i + 1) % 17 != 0 else '\n' )

AnnData object with n_obs × n_vars = 1 × 58347 
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   


Now that you've set up your two matrices in scanpy, you're ready to perform whatever analysis piques your interest.